In [1]:
from nltk.tokenize import sent_tokenize
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import json
import time
import re

In [5]:
# df_kospi_filing_2011_2015 = pd.read_excel('C:\\Users\\sylim2357\\Documents\\SNU\\projects\\10-KPlanet\\kospi200_2011_2015.xlsx')
# df_kospi_filing_2011_2015 = pd.read_excel('C:\\Users\\sylim2357\\Documents\\SNU\\FVID\\10k delta\\Data\\kospi200_2011_2015.xlsx')

df_kospi_filing_2015_2019 = pd.read_excel(r'C:\Users\Elaine\Desktop\kospi200_2015_2019.xlsx')

In [6]:
# Use nltk to split sentences (works great on Korean as well)
raw_text = df_kospi_filing_2015_2019['Text']
token_text = raw_text.apply(lambda x: sent_tokenize(x))

# Use extra hand-made parser
def additional_sent_tokenize(text):
    '''
    Some bullet points, which should be taken as separate sentences,
    are not accounted for by the nltk sentence tokenizer.
    This additional tokenizer will take care of such bullet points.
    '''
    if '※' in text:
        text_list = text.split('※')
    elif '* ' in text:
        text_list = text.split('* ')
    else:
        text_list = [text]
    return text_list

additional_token_text =[]

for t in token_text:
    temp_list = []
    for sent in t:
        temp_list.append(additional_sent_tokenize(sent))
    additional_token_text.append([item for sublist in temp_list for item in sublist])

# Select relevant, proper sentences and pre-process
def clean_text(text):
    '''
    Clean the messy raw text
    1) Remove \n, 전자공시시스템 dart.fss.or.kr, Page #
    2) Remove white space before and after the input text
    '''
    text = re.sub("\\n|(전자공시시스템 dart\.fss\.or\.kr)?(\n)?(Page \d{1,2})?(?=\s)",
                  " ",
                  text)
    text = re.sub("\s\s+", " ", text).strip()
    return text

sent_text = []

for t in additional_token_text:
    relevant_sentences = []
    
    for s in t:
        if s.endswith('니다.'):
            s = clean_text(s)
            s = s.strip()
            if len(s) > 5000:
                split_txt = [s[i*len(s)//30:(i+1)*len(s)//30] for i in range(30)]
                relevant_sentences += split_txt
            if len(s) > 3000:
                split_txt = [s[i*len(s)//20:(i+1)*len(s)//20] for i in range(20)]
                relevant_sentences += split_txt
            if len(s) > 1000:
                split_txt = [s[i*len(s)//10:(i+1)*len(s)//10] for i in range(10)]
                relevant_sentences += split_txt
            else:
                relevant_sentences += [s]
            
    sent_text.append(relevant_sentences)

# Save checkpoint
df_kospi_filing_2015_2019['Relevant_Sentences'] = sent_text

In [7]:
df_kospi_filing_2015_2019_parsed = df_kospi_filing_2015_2019.drop(['Text'], axis=1)
# df_kospi_filing_2015_2019_parsed.to_csv('kospi200_2015_2019_parsed.csv', encoding='utf-8', index=False)

In [8]:
def make_sublist(text):
    batch = []
    batch_size = 5
    num_batches = int(np.ceil(len(text)/batch_size))
    for i in range(num_batches):
        batch.append(' '.join(text[batch_size*i:batch_size*(i+1)]))
    return batch

text_series_sublists = df_kospi_filing_2015_2019_parsed['Relevant_Sentences'].apply(make_sublist)

In [9]:
#path = "C:\\Users\\sylim2357\\Documents\\chromedriver_win32\\chromedriver.exe" # nathan's path

path = r'C:\Users\Elaine\Desktop\chromedriver.exe' # elaine's path
driver = webdriver.Chrome(path)
driver.get('https://translate.google.com/#view=home&op=translate&sl=ko&tl=en')
elem = driver.find_element_by_id("source")
df_kospi_filing_2015_2019_parsed['Translated'] = ''
# translated_text_list = []
for idx, text_sublist in enumerate(text_series_sublists):
    print(str(idx)+': translating')
    translated_text = []
    for text in text_sublist:
        elem.clear()
        time.sleep(np.random.randint(50,300)/101)
        elem.send_keys(text)
        status = True
        while status:
            time.sleep(np.random.randint(50,300)/101)
            if len(driver.find_elements_by_class_name("tlid-translation")) > 0:
                status = False
        translated_text.append(driver.find_element_by_class_name("tlid-translation").text)
    print(str(idx)+': translated')
#     translated_text_list.append('. '.join(translated_text))
    df_kospi_filing_2015_2019_parsed.loc[idx, 'Translated'] = ' '.join(translated_text)
    if idx % 500 == 0:
        df_kospi_filing_2015_2019_parsed.to_csv('kospi200_2015_2019_translated.csv', encoding='utf-8', index=False)
driver.close()
df_kospi_filing_2015_2019_parsed.to_csv('kospi200_2015_2019_translated.csv', encoding='utf-8', index=False)

0: translating
0: translated
1: translating
1: translated
2: translating
2: translated
3: translating
3: translated
4: translating
4: translated
5: translating
5: translated
6: translating
6: translated
7: translating
7: translated
8: translating
8: translated
9: translating
9: translated
10: translating
10: translated
11: translating
11: translated
12: translating
12: translated
13: translating
13: translated
14: translating
14: translated
15: translating
15: translated
16: translating
16: translated
17: translating
17: translated
18: translating
18: translated
19: translating
19: translated
20: translating
20: translated
21: translating
21: translated
22: translating
22: translated
23: translating
23: translated
24: translating
24: translated
25: translating
25: translated
26: translating
26: translated
27: translating
27: translated
28: translating
28: translated
29: translating
29: translated
30: translating
30: translated
31: translating
31: translated
32: translating
32: translat

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.122)


In [9]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=80.0.3987.122)
